In [1]:
import Pkg; 

cd(joinpath(@__DIR__, "../../"))
Pkg.activate("Project.toml")

using MorphoMol
using StaticArrays
using Rotations
using Distances
using Distributions
using JLD2

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolHelpers`


In [2]:
@load "../../Data/jld2/single_subunits/6r7m_protor.jld2"

3-element Vector{Symbol}:
 :template_mol
 :template_radii
 :x_init

In [7]:
function get_template_values(mol_type)
    @load "../../Data/jld2/single_subunits/$(mol_type)_protor.jld2" template_mol template_radii x_init
    (template_mol, template_radii)
end

templates = Dict(mol_type => get_template_values(mol_type) for mol_type in ["6r7m", "6sae", "6sag", "1ei7", "2tmv"])

open("src/template_data.jl", "w") do file
    write(file, "templates = $(templates)")
end

380659

In [2]:
id = "1stm"
@load "assets/input/jld2/single_subunits/$(id)_protor.jld2" template_mol template_radii x_init
n_mol = 5
n_atoms_per_mol = length(template_radii)
radii = vcat([template_radii for i in 1:n_mol]...);
mkpath("assets/input/jld2/$(n_mol)_$(id)_init/")


"assets/input/jld2/5_1stm_init"

In [17]:
size(template_mol)[2]

1061

In [15]:
i = 1
for x in eachcol(exp(Rotations.RotationVecGenerator(x_init[i:i+2]...)) * template_mol .+ x_init[i+3:i+5])
    @SVector[x[1], x[2], x[3]]
end

In [5]:
template_mol

3×1061 Matrix{Float64}:
 -28.2256  -26.9189   -26.4582   …  -34.1016   -33.1148   -34.9277
  11.3593   10.803      9.75629      -5.56976   -4.64423   -8.47959
  -3.0019   -3.06405   -2.06098      11.5434    11.9563    11.2767

In [23]:
bounds = 120.0
for i in 1:50
    x_init = vcat([
        [rand(Uniform(0.0, 2*pi)), rand(Uniform(0.0, 2*pi)), rand(Uniform(0.0, 2*pi)), 
        rand(Uniform(0.0, bounds)), rand(Uniform(0.0, bounds)), rand(Uniform(0.0, bounds))] 
        for i in 1:n_mol]...);
        
    @save "assets/input/jld2/$(n_mol)_$(id)_init/$(id)_protor_$(i).jld2" template_mol template_radii x_init
end

In [27]:
hotstart_folder = "assets/hpc_out_d/5_1stm_oj_0_0_os_0_85/"
for file in readdir(hotstart_folder)
    if split(file, ".")[end] == "jld2"
        @load "$hotstart_folder$file" in_out_data
        mindex = argmin(in_out_data.output.Es)
        x_init = in_out_data.output.states[mindex]
        id = parse(Int, split(file, "_")[3])
        template_mol = in_out_data.input.template_mol
        template_radii = in_out_data.input.template_radii
        @save "assets/input/jld2/5_1stm_hotstart/1stm_protor_$(id).jld2" template_mol template_radii x_init
    end
end

In [5]:
@load "assets/input/jld2/2_6r7m_touching_init/2_6r7m_touching_1.jld2" template_mol template_radii x_init
for i in 51:100
    @save "assets/input/jld2/2_6r7m_touching_init/2_6r7m_touching_$(i).jld2"
end